## Algoritmo de Flajolet-Martin

La idea detrás de este algoritmo es que entre más elementos diferentes haya en el flujo de datos, más valores hash diferentes veremos y es más probable que alguno de estos valores tenga una representación binaria que termine con un mayor número de ceros consecutivos.

In [2]:
import numpy as np
np.random.seed(2021) # para reproducibilidad

class ConteoProbabilista:  
    def __init__(self, n_cubetas, primo, n_bits=64):
        self.primo = primo  
        self.n_cubetas = n_cubetas
        self.a = np.random.randint(1, self.primo - 1)
        self.b = np.random.randint(0, self.primo - 1)
        self.bitmap = np.zeros(n_bits, dtype=np.bool)

    def __call__(self, x):
        hv = ((self.a * x + self.b) % self.primo) % self.n_cubetas
        i = bin(hv)[2:][::-1].find('1')
        self.bitmap[i] = 1

    def cardinalidad(self):
        r = np.argwhere(self.bitmap == 0)[0]
        return (2**r) / 0.77351

Definimos una clase que realiza varias estimaciones, las divide en grupos pequeños, obtiene la mediana de las estimaciones de cada grupo y toma el promedio de las medianas como estimación final.

In [3]:
class EstimadorElementosDistintos:
    def __init__(self, n_cubetas, n_grupos, n_funciones, primo, n_bits):
        self.n_grupos = n_grupos
        self.n_funciones = n_funciones
        self.estimadores = []
        for i in range(self.n_grupos):
            func = []
            for j in range(self.n_funciones):
                func.append(ConteoProbabilista(n_cubetas, primo, n_bits))
            self.estimadores.append(func)
        self.conteos = np.zeros((self.n_grupos, self.n_funciones))

    def __call__(self, x):
        for i in range(self.n_grupos):
            for j in range(self.n_funciones):  
                self.estimadores[i][j](x)
                self.conteos[i, j] = self.estimadores[i][j].cardinalidad()

    def cardinalidad(self):
        return np.mean(np.median(self.conteos, axis=1))

Generamos números aleatorios.

In [4]:
X = np.random.randint(0,100000, size=1000000)
print("Hay {0} elementos distintos".format(np.unique(X).size))

Hay 99995 elementos distintos


Instanciamos nuestra clase y estimamos elementos distintos.

In [5]:
est = EstimadorElementosDistintos(10000000, 5, 10, 4294967291, 64)

for i,x in enumerate(X):
    est(x)
  
print(u'Real = {0} Estimación = {1} '.format(np.unique(X[:i+1]).size,est.cardinalidad()))

/tmp/ipykernel_9532/529691460.py:10: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self.bitmap = np.zeros(n_bits, dtype=np.bool)


Real = 99995 Estimación = 84725.47219816162 


In [6]:
est

In [8]:
est.conteos

array([[ 42362.73609908,  84725.47219816,   2647.67100619,
        338901.88879265,  42362.73609908,  10590.68402477,
        169450.94439632,  84725.47219816,  84725.47219816,
        338901.88879265],
       [ 84725.47219816,  84725.47219816,  21181.36804954,
         84725.47219816,  84725.47219816,  42362.73609908,
        169450.94439632,  84725.47219816,  84725.47219816,
        169450.94439632],
       [ 84725.47219816, 338901.88879265,  21181.36804954,
         84725.47219816,  84725.47219816,  84725.47219816,
         42362.73609908, 169450.94439632,  10590.68402477,
        338901.88879265],
       [  5295.34201239,  42362.73609908,  84725.47219816,
         84725.47219816,  42362.73609908,  21181.36804954,
         42362.73609908,  10590.68402477, 169450.94439632,
        169450.94439632],
       [169450.94439632, 169450.94439632,  84725.47219816,
         84725.47219816,  21181.36804954,  84725.47219816,
        169450.94439632, 169450.94439632,  84725.47219816,
        169